## Agile Hardware Design
***
# Sequential Circuits

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Spring21/)

## Plan for Today

* Registers!
* Example sequential blocks
* State machines
* Viewing simulation results

## Loading The Chisel Library Into a Notebook

In [1]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling /Users/sbeamer/Spring 2021/CSE 293/lectures/04-state/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /Users/sbeamer/Spring 2021/CSE 293/lectures/04-state/Main.sc #2

path: String = "/Users/sbeamer/Spring 2021/CSE 293/lectures/04-state/../resource/chisel_deps.sc"

In [35]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import treadle._

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

import treadle._

## Registers

* Explicitly declare a register with `Reg(type)`
  * Contrast from Verilog where registers are synthesized
* A register is simply another component, no special semantics for time
  * Need to connect its input and output
* `clock` and `reset` are implicit
* Not covered today: functionality for using other clocks ([multi-clock](https://www.chisel-lang.org/chisel3/docs/explanations/multi-clock.html) and [asynchronous reset](https://www.chisel-lang.org/chisel3/docs/explanations/reset.html)

<img src="images/reg.svg" alt="register schamic" style="width:100%;" align="center"/>

## Other Flavors of `Reg`

* Helper functions streamline

### [Set Initial Value](https://www.chisel-lang.org/api/latest/chisel3/RegInit$.html) - `RegInit(init)`
* Value applied synchronously when `reset` true

### [Attach Input](https://www.chisel-lang.org/api/latest/chisel3/RegNext$.html) - `RegNext(next)`
* Saves a connect statement for input
* Useful for delaying a signal by a cycle

### [Enable](https://www.chisel-lang.org/api/latest/chisel3/util/RegEnable$.html) - `RegEnable(next, en)`
* Write enable for when to update

In [9]:
class RegLand extends Module {
    val io = IO(new Bundle {
        val in  = Input(Bool())
        val en  = Input(Bool())
        val out = Output(Bool())
    })
//     val r = Reg(Bool())
//     val r = RegInit(0.B)
//     r := io.in
//     io.out := r
//     io.out := RegNext(io.in, 0.B)
    io.out := RegEnable(io.in, 0.B, io.en)
}
println(getVerilog(new RegLand))

Elaborating design...
Done elaborating.
module RegLand(
  input   clock,
  input   reset,
  input   io_in,
  input   io_en,
  output  io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg  r; // @[Reg.scala 27:20]
  assign io_out = r; // @[cmd8.sc 12:12]
  always @(posedge clock) begin
    if (reset) begin // @[Reg.scala 27:20]
      r <= 1'h0; // @[Reg.scala 27:20]
    end else if (io_en) begin // @[Reg.scala 28:19]
      r <= io_in; // @[Reg.scala 28:23]
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
`ifndef SYNTHESIS
`ifdef FIRRTL_BEFORE_INITIAL
`FIRRTL_BEFORE_INITIAL
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
  

defined class RegLand

## Example: Counter (done manually)

In [18]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = Reg(UInt(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    val applyEn = Mux(io.en, nextVal, count)
    count := Mux(reset.asBool, 0.U, applyEn)
    io.out := count
}
println(getVerilog(new MyCounter(15)))

Elaborating design...
Done elaborating.
module MyCounter(
  input        clock,
  input        reset,
  input        io_en,
  output [3:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [3:0] count; // @[cmd17.sc 6:20]
  wire [3:0] _T_2 = count + 4'h1; // @[cmd17.sc 7:47]
  assign io_out = count; // @[cmd17.sc 10:12]
  always @(posedge clock) begin
    if (reset) begin // @[cmd17.sc 9:17]
      count <= 4'h0;
    end else if (io_en) begin // @[cmd17.sc 8:22]
      if (count < 4'hf) begin // @[cmd17.sc 7:22]
        count <= _T_2;
      end else begin
        count <= 4'h0;
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif

defined class MyCounter

## Example: Counter (use RegInit)

In [19]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    val nextVal = Mux(count < maxVal.U, count + 1.U, 0.U)
    count := Mux(io.en, nextVal, count)
    io.out := count
}
println(getVerilog(new MyCounter(15)))

Elaborating design...
Done elaborating.
module MyCounter(
  input        clock,
  input        reset,
  input        io_en,
  output [3:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [3:0] count; // @[cmd18.sc 6:24]
  wire [3:0] _T_2 = count + 4'h1; // @[cmd18.sc 7:47]
  assign io_out = count; // @[cmd18.sc 9:12]
  always @(posedge clock) begin
    if (reset) begin // @[cmd18.sc 6:24]
      count <= 4'h0; // @[cmd18.sc 6:24]
    end else if (io_en) begin // @[cmd18.sc 8:17]
      if (count < 4'hf) begin // @[cmd18.sc 7:22]
        count <= _T_2;
      end else begin
        count <= 4'h0;
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  inte

defined class MyCounter

## Example: Counter (using when)

In [24]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            count := count + 1.U
        } .otherwise {
            count := 0.U
        }
    }
    io.out := count
}
println(getVerilog(new MyCounter(15)))

Elaborating design...
Done elaborating.
module MyCounter(
  input        clock,
  input        reset,
  input        io_en,
  output [3:0] io_out
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [3:0] count; // @[cmd23.sc 6:24]
  wire [3:0] _T_2 = count + 4'h1; // @[cmd23.sc 9:28]
  assign io_out = count; // @[cmd23.sc 14:12]
  always @(posedge clock) begin
    if (reset) begin // @[cmd23.sc 6:24]
      count <= 4'h0; // @[cmd23.sc 6:24]
    end else if (io_en) begin // @[cmd23.sc 7:18]
      if (count < 4'hf) begin // @[cmd23.sc 8:33]
        count <= _T_2; // @[cmd23.sc 9:19]
      end else begin
        count <= 4'h0; // @[cmd23.sc 11:19]
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $rand

defined class MyCounter

## Example: Counter (too dense?)

In [23]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val count = Output(UInt(log2Ceil(maxVal+1).W))
    })
    io.count := RegEnable(Mux(io.count < maxVal.U, io.count + 1.U, 0.U), 0.U, io.en)
}
println(getVerilog(new MyCounter(15)))

Elaborating design...
Done elaborating.
module MyCounter(
  input        clock,
  input        reset,
  input        io_en,
  output [3:0] io_count
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  wire [3:0] _T_2 = io_count + 4'h1; // @[cmd22.sc 6:61]
  reg [3:0] r; // @[Reg.scala 27:20]
  assign io_count = r; // @[cmd22.sc 6:14]
  always @(posedge clock) begin
    if (reset) begin // @[Reg.scala 27:20]
      r <= 4'h0; // @[Reg.scala 27:20]
    end else if (io_en) begin // @[Reg.scala 28:19]
      if (io_count < 4'hf) begin // @[cmd22.sc 6:30]
        r <= _T_2;
      end else begin
        r <= 4'h0;
      end
    end
  end
// Register and memory initialization
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  intege

defined class MyCounter

## Testing MyCounter

In [36]:
test(new MyCounter(3), Seq(WriteVcdAnnotation)) { c =>
// test(new MyCounter(3)) { c =>
    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(1.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(2.U)
    c.clock.step()

    c.io.en.poke(0.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(3.U)
    c.clock.step()

    c.io.en.poke(1.B)
    c.io.out.expect(0.U)
    c.clock.step()
}

Elaborating design...
Done elaborating.
test MyCounter Success: 0 tests passed in 8 cycles in 0.046203 seconds 173.15 Hz


## Chisel `Enum`

* `Enum` is Chisel object that assigns `UInt`s (`Enumeration` is Scala)
```scala
val nameA :: nameB :: nameC :: Nil = Enum(3)
```
* Helpful for putting human-sensical names to distinct values
* Example use cases
  * Naming states in a state machine
  * Labeling mux way selection options
  * Labeling options for interfaces
* `ChiselEnum` is a new mechanism in [experimental](https://www.chisel-lang.org/chisel3/docs/explanations/chisel-enum.html)

## Example State Machine (Raccoon)

<img src="images/raccoon.svg" alt="raccoon fsm" style="width:45%;" align="center"/>

In [28]:
class Raccoon extends Module { // using when
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(UInt())
    })
    val hide :: wander :: rummage :: eat :: Nil = Enum(4)
    val state = RegInit(hide)
    when (state === hide) {
        when (!io.noise) { state := wander }
    } .elsewhen (state === wander) {
        when (io.noise) { state := hide }
        .elsewhen (io.trash) { state := rummage }
    } .elsewhen (state === rummage) {
        when (io.noise) { state := hide }
        .elsewhen (io.food) { state := eat }
    } .elsewhen (state === eat) {
        when (io.noise) { state := hide }
        .elsewhen (!io.food) { state := wander }
    }
    io.action := state
}
println(getVerilog(new Raccoon))

Elaborating design...
Done elaborating.
module Raccoon(
  input        clock,
  input        reset,
  input        io_noise,
  input        io_trash,
  input        io_food,
  output [1:0] io_action
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [1:0] state; // @[cmd27.sc 9:24]
  wire [1:0] _GEN_1 = io_trash ? 2'h2 : state; // @[cmd27.sc 14:30 cmd27.sc 14:38 cmd27.sc 9:24]
  wire [1:0] _GEN_3 = io_food ? 2'h3 : state; // @[cmd27.sc 17:29 cmd27.sc 17:37 cmd27.sc 9:24]
  wire [1:0] _GEN_4 = io_noise ? 2'h0 : _GEN_3; // @[cmd27.sc 16:25 cmd27.sc 16:33]
  wire [1:0] _GEN_5 = ~io_food ? 2'h1 : state; // @[cmd27.sc 20:30 cmd27.sc 20:38 cmd27.sc 9:24]
  wire [1:0] _GEN_6 = io_noise ? 2'h0 : _GEN_5; // @[cmd27.sc 19:25 cmd27.sc 19:33]
  wire [1:0] _GEN_7 = state == 2'h3 ? _GEN_6 : state; // @[cmd27.sc 18:33 cmd27.sc 9:24]
  assign io_action = state; // @[cmd27.sc 22:15]
  always @(posedge clock) begin
    if (reset) begin // @[cmd27.sc 9:24]
      state <

defined class Raccoon

In [32]:
class Raccoon extends Module { // using switch
    val io = IO(new Bundle {
        val noise = Input(Bool())
        val trash = Input(Bool())
        val food  = Input(Bool())
        val action = Output(UInt())
    })
    val hide :: wander :: rummage :: eat :: Nil = Enum(4)
    val state = RegInit(hide)
    switch(state) {
        is (hide) {
            when (!io.noise) { state := wander }
        }
        is (wander) {
            when (io.noise) { state := hide }
            .elsewhen (io.trash) { state := rummage }
        }
        is (rummage) {
            when (io.noise) { state := hide }
            .elsewhen (io.food) { state := eat }
        }
        is (eat) {
            when (io.noise) { state := hide }
            .elsewhen (!io.food) { state := wander }
        }
    }
    io.action := state
}
println(getVerilog(new Raccoon))

Elaborating design...
Done elaborating.
module Raccoon(
  input        clock,
  input        reset,
  input        io_noise,
  input        io_trash,
  input        io_food,
  output [1:0] io_action
);
`ifdef RANDOMIZE_REG_INIT
  reg [31:0] _RAND_0;
`endif // RANDOMIZE_REG_INIT
  reg [1:0] state; // @[cmd31.sc 9:24]
  wire  _T = 2'h0 == state; // @[Conditional.scala 37:30]
  wire  _T_2 = 2'h1 == state; // @[Conditional.scala 37:30]
  wire [1:0] _GEN_1 = io_trash ? 2'h2 : state; // @[cmd31.sc 16:34 cmd31.sc 16:42 cmd31.sc 9:24]
  wire  _T_3 = 2'h2 == state; // @[Conditional.scala 37:30]
  wire [1:0] _GEN_3 = io_food ? 2'h3 : state; // @[cmd31.sc 20:33 cmd31.sc 20:41 cmd31.sc 9:24]
  wire [1:0] _GEN_4 = io_noise ? 2'h0 : _GEN_3; // @[cmd31.sc 19:29 cmd31.sc 19:37]
  wire  _T_4 = 2'h3 == state; // @[Conditional.scala 37:30]
  wire [1:0] _GEN_5 = ~io_food ? 2'h1 : state; // @[cmd31.sc 24:34 cmd31.sc 24:42 cmd31.sc 9:24]
  wire [1:0] _GEN_6 = io_noise ? 2'h0 : _GEN_5; // @[cmd31.sc 23:29 cm

defined class Raccoon

In [34]:
test(new Raccoon()) { r =>
    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)

    r.io.noise.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.trash.poke(1.B)
    r.clock.step()
    r.io.action.expect(2.U)

    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(1.B)
    r.clock.step()
    r.io.action.expect(0.U)
}

Elaborating design...
Done elaborating.
test Raccoon Success: 0 tests passed in 9 cycles in 0.006980 seconds 1289.35 Hz


In [ ]:
test(new Raccoon()) { r =>
    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(1.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(2.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(1.B)
    r.clock.step()
    r.io.action.expect(3.U)

    r.io.noise.poke(0.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(1.U)

    r.io.noise.poke(1.B)
    r.io.trash.poke(0.B)
    r.io.food.poke(0.B)
    r.clock.step()
    r.io.action.expect(0.U)
}

## Collecting Useful Output from Simulation

* So far, we have used a PeekPoke tester to interact with the design
    * Running tests is specific form of simulation
    * Tests can explicitly check for certain behavior
* Can set up explicit print statements (next slide)
* Can record waveforms as VCD (demo)
* Recommend building extensive tests, use prints & waveforms for debugging (but not tests)

## Printing

### In Scala (during generation)
* Can use `println`
* Scala string interpolation allows for variable names or even expressions inside strings
```scala
println(s"this is $myVal and ${foo.bar}")
```

### In Chisel (during simulation)
* Can use `printf` in C-style
```scala
printf("myVal: %d", myVal)
```
* ... or interpolation style
```scala
printf(p"myVal: $myVal")
```

## Additional Ways to Express Literals

* Can add `.U`, `.S`, `.B` to cast to `UInt`, `SInt`, `Bool`
* Can specify bitwidth with more explicit types
* Can prefix a string with encoding `h` (hex), `o` (octal), `b` (binary)
* Can break up long literals with `_`

In [ ]:
"ha".U
"h_dead_beef".U
"ha".U
"ha".U(8.W)
"ha".asUInt(8.W)